Uso de Gradio para la interfaz de visualización de valores atípicos / anomalías (outliers).

In [ ]:
# Importación de librerías
import pandas as pd
import numpy as np
import glob
import gradio as gr
import seaborn as sns
import matplotlib.pyplot as plt

# Lectura de archivos CSV
path = "samples/"
all_files = glob.glob(path + "/*.csv")
df_list = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df_list.append(df)

# Unión de todos los archivos en un solo dataframe y reemplazo de valores faltantes con "UNK".
df_concat = pd.concat(df_list, axis=0, ignore_index=True)
df_concat["Tipo de alerta"] = df_concat["Tipo de alerta"].fillna("UNK")


# Función para calcular estadísticas y visualizar valores atípicos
def ver_outliers_stats_y_limites(codigo, tipo_metrica, tipo_alerta):
    # Filtración de los valores específicos
    filtered_df = df_concat[
        (df_concat["Codigo"] == codigo)
        & (df_concat["Clasificacion de metrica"] == tipo_metrica)
        & (df_concat["Tipo de alerta"] == tipo_alerta)
    ]

    # Cálculo para detectar valores atípicos (outliers) en la columna Valor
    q1 = filtered_df["Valor"].quantile(0.25)
    q3 = filtered_df["Valor"].quantile(0.75)
    iqr = q3 - q1
    limite_inferior = q1 - 1.5 * iqr
    limite_superior = q3 + 1.5 * iqr

    # Encontrando los valores exactos de las barbas (whiskers) de la caja / box plot
    whisker_low = filtered_df[filtered_df["Valor"] >= limite_inferior]["Valor"].min()
    whisker_high = filtered_df[filtered_df["Valor"] <= limite_superior]["Valor"].max()

    stats = {
        "Media": filtered_df["Valor"].mean(),
        "Mediana": filtered_df["Valor"].median(),
        "Q1": q1,
        "Q3": q3,
        "IQR": iqr,
        "Mínimo": filtered_df["Valor"].min(),
        "Máximo": filtered_df["Valor"].max(),
        "Límite inferior calculado": limite_inferior,
        "Límite superior calculado": limite_superior,
        "Barba inferior (whisker_low)": whisker_low,
        "Barba superior (whisker_high)": whisker_high,
    }
    # Colocando las stats calculadas de forma legible para la interfaz gráfica
    stats_text = "\n".join([f"{key}: {value:.2f}" for key, value in stats.items()])

    # Creando la nueva figura con dimensiones específicas
    plt.figure(figsize=(10, 6))

    # Creación del box plot con Seaborn
    sns.boxplot(y=filtered_df["Valor"])

    # Ilustración de límites calculados
    plt.axhline(
        y=limite_inferior,
        color="r",
        linestyle="--",
        label="Límite inferior (calculado)",
    )
    plt.axhline(
        y=limite_superior,
        color="b",
        linestyle="--",
        label="Límite superior (calculado)",
    )
    plt.legend()

    # Personalizando la figura creada plt
    plt.title(f"Outliers for {codigo}, {tipo_metrica}, {tipo_alerta}")
    plt.ylabel("Valor")
    plt.grid(True)

    return plt, stats_text


# Creación de las entradas como dropdowns
codigo_dropdown = gr.Dropdown(list(df_concat["Codigo"].unique()), label="Código")
clasificacion_metrica_dropdown = gr.Dropdown(
    list(df_concat["Clasificacion de metrica"].unique()),
    label="Clasificacion de métrica",
)
tipo_alerta_dropdown = gr.Dropdown(
    list(df_concat["Tipo de alerta"].unique()), label="Tipo de alerta"
)

# Creación de la interfaz con Gradio
demo = gr.Interface(
    fn=ver_outliers_stats_y_limites,
    inputs=[codigo_dropdown, clasificacion_metrica_dropdown, tipo_alerta_dropdown],
    outputs=["plot", "text"],
    title="Visualizar valores atípicos (outliers), estadísticas, y límites",
    description="Seleccione un código, una clasificación de métrica, y un tipo de alerta para visualizar outliers, ver estadísticas clave, y entender los límites que definen los outliers.",
)

# Desplegando la interfaz
demo.launch()

In [ ]:
# Deteniendo la demostración
demo.close()